# XGBoost trainer

This notebook function handles training and logging of xgboost models **only**, exposing both the sklearn and low level api's.

## steps
1. generate an xgboost model configuration by selecting one of 5 available types
2. get a sample of data from a data source (random rows, consecutive rows, or the entire dataset)
3. split the data into train, validation, and test sets.  

> _PLEASE NOTE_:  there are many approaches to cross validation (cv) and as many ways to implement cv in scikit learn.  In this third stage, an alternative, two-way train and test split can be created.  The training set would then, for example, serve as input to a cross validation splitter.  The latter creates multiple training and validation subsets, called folds. These folds are then input, either in sequence or in parallel into the fit algorithm.

4. train the model
5. dump the model
6. generate predictions and probabilities
7. calibrate probabilities if needed
8. calculate evaluation statistics and plots

All these steps have been separated here into independent functions since many can be reused for other model types. Some of the following functions will be transferred in the `mlrun.mlutils` module. Additionally, each function contains its own imports in order to isolate and identify dependencies.

In [1]:
# nuclio: ignore
import nuclio

In [2]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

## generate an xgb model

In [3]:
def gen_xgb_model(model_type: str, xgb_params: dict):
    """generate an xgboost model
    
    Multiple model types that can be estimated using
    the XGBoost Scikit-Learn API
    
    :param model_type: one of "classifier", "regressor",
                       "ranker", "rf_classifier", or
                      "rf_regressor"
    :param xgb_params: parameters passed through the 
                       function execution context
    """
    from json import load
    from mlrun.mlutils import get_class_fit, create_class

    # generate model and fit function
    mtypes = {
        "classifier"   : "xgboost.XGBClassifier",
        "regressor"    : "xgboost.XGBRegressor",
        "ranker"       : "xgboost.XGBRanker",
        "rf_classifier": "xgboost.XGBRFClassifier",
        "rf_regressor" : "xgboost.XGBRFRegressor"
    }
    if model_type not in mtypes.keys():
        raise Exception("unrecognized model types, see help documentation")
    
    model_config = get_class_fit(mtypes[model_type])

    for k, v in xgb_params:
        if k.startswith("CLASS_"):
            model_config["CLASS"][k[6:]] = v
        if k.startswith("FIT_"):
            model_config["FIT"][k[4:]] = v

    ClassifierClass = create_class(model_config["META"]["class"])
    model = ClassifierClass(**model_config["CLASS"])

    return model, model_config

## get a sample of data

In [4]:
def get_sample(src:str, sample: int, label: str, reader=None):
    """generate data sample to be split (candidate for mlrun)
     
    Returns features matrix and header (x), and labels (y)
    :param src:    full path and filename of data artifact
    :param sample: sample size from data source, use negative 
                   integers to sample randomly, positive to
                   sample consecutively from the first row
    :param label:  label column title
    :param reader: pandas type reader (read_csv, read_parquet, ...) returning
                   a pandas dataframe, and with a `dropna` attribute
    """
    import pandas as pd
    
    # read data function -- deprecate/add new mlrun dataset functionality
    if not reader:
        if src.endswith("csv"):
            reader = pd.read_csv
        elif src.endswith("parquet") or src.endswith("pq"):
            reader = pd.read_parquet
        else:
            raise Exception(f"file type unhandled {src}")

    # get sample
    if (sample == -1) or (sample >= 1):
        # get all rows, or contiguous sample starting at row 1.
        raw = reader(src).dropna()
        labels = raw.pop(label)
        raw = raw.iloc[:sample, :]
        labels = labels.iloc[:sample]
    else:
        # grab a random sample
        raw = reader(src).dropna().sample(sample * -1)
        labels = raw.pop(label)

    return raw, labels, raw.columns.values

## split data into train, validate and test

In [5]:
def get_splits(
    raw, 
    labels, 
    n_ways: int = 3,
    test_size: float = 0.15,
    valid_size: float = 0.30,
    label_names: list = ["labels"],
    random_state: int = 1
):
    """generate train and test sets (candidate for mlrun)

    cross validation:
    1. cut out a test set
    2a. use the training set in a cross validation scheme, or
    2b. make another split to generate a validation set
    
    2 parts (n_ways=2): train and test set only
    3 parts (n_ways=3): train, validation and test set
    4 parts (n_ways=4): n_ways=3 + a held-out probability calibration set
    
    :param raw:            dataframe or numpy array of raw features
    :param labels:         dataframe or numpy array of raw labels
    :param n_ways:         (3) split data into 2, 3, or 4 parts
    :param test_size:      proportion of raw data to set asid as test data
    :param valid_size:     proportion of remaining data to be set as validation
    :param label_names:         label names
    :param random_state:   (1) random number seed
    """
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    
    if isinstance(raw, np.ndarray):
        if labels.ndim==1:
            labels=labels.reshape(-1,1)
        xy = np.concatenate([raw, labels], axis=1)
    else:
        if isinstance(labels, pd.Series):
            labels = pd.DataFrame(data=labels, columns=label_names)
        xy = pd.concat([raw, labels], axis=1)
        
    x, xte, y, yte = train_test_split(xy, labels, test_size=test_size,
                                      random_state=random_state)
    if n_ways==2:
        return (x, y), (xte, yte), None, None
    elif n_ways==3:
        xtr, xva, ytr, yva = train_test_split(x, y,train_size=valid_size,
                                              random_state=random_state)
        return (xtr, ytr), (xva, yva), (xte, yte), None
    elif n_ways==4:
        xt, xva, yt, yva = train_test_split(x, y,train_size=valid_size,
                                              random_state=random_state)
        xtr, xcal, ytr, ycal = train_test_split(xt, yt, train_size=0.8,
                                              random_state=random_state)
        return (xtr, ytr), (xva, yva), (xte, yte), (xcal, ycal)
    else:
        raise Exception("n_ways must be in the range [2,4]")

## save the test data separately


In [6]:
def save_test_set(
    context, 
    xtest, 
    ytest, 
    header: list, 
    label: str = "labels", 
    file_ext: str = "parquet", 
    index: bool = False,
    debug: bool = False
):
    """log a held out test set

    :param context:    the function execution context
    :param xtest:      test features, as np.ndarray output from `get_splits`
    :param ytest:      test labels, as np.ndarray output from `get_splits`
    :param header:     ([])features header if required
    :param label:      ("labels") name of label column
    :param file_ext:   format of test set file
    :param index:      preserve index column
    :param debug:      (False)
    """
    import pandas as pd
    from mlrun import mlconf
    
    test_set = pd.concat(
        [pd.DataFrame(data=xtest, columns=header),
         pd.DataFrame(data=ytest.values, columns=[label])],
        axis=1,)
    
    if debug:
        test_set.to_parquet(mlconf.artifact_path+'/test_set-dev.parquet')
        
    context.log_dataset("test_set", df=test_set, format=file_ext, index=index)

## dump an xgb model

In [7]:
def dump_xgb_model(
    context, 
    model,
    dump_type: str,
    dest_folder: str,
    dest_name: str
):
    """serialize/log model
    
    XGBoost model can be save in 3 different ways:
    1. pickle the internal _booster object, inside the model
    2. using model.save_model("fn.bin") using a legacy binary xgb format
    2. using model.save_model("fn.json") using a portable json format
    
    :param context:     the function"s execution context
    :param model:       the fitted xgboost model
    :param dump_type:   "pickle" legacy", or "json", 
    :param dest_folder: path for serialized model 
    :param dest_name:   name for serialized model file
    """
    from cloudpickle import dumps, dump
    try:
        # if dump_type is "pickle":
        # https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier.save_model
        model.save_model(f"{dest_folder}/{dest_name}-save_model.pkl")
        
        # elif dump_type is "json":
        # see https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html
        # this save all contents as json
        model.save_model(f"{dest_folder}/{dest_name}-save_model.json")
        
        # else:
        # this saves all internal contents as pickle
        _booster = model.get_booster()
        dump(_booster, open(f"{dest_folder}/{dest_name}-dump.pkl", "wb"))
        dump(model, open(f"{dest_folder}/{dest_name}-dump-model.pkl", "wb"))
        
        # log model needs to be spec"ed:
        data = dumps(_booster)
        context.log_artifact("model", body=data, local_path=f"{dest_folder}/{dest_name}.pkl")
    except Exception as e:
        print("xgboost model serialization error", str(e))

## plots

In [8]:
def plot_confusion_matrix(
    labels,
    predictions,
    classes,
    normalize="all",
    title='Confusion matrix',
    cmap=None
):
    """prints and plots the confusion matrix.
    
    """
    import matplotlib.pyplot as plt
    from sklearn import metrics
    import numpy as np
    import itertools
    
    if not cmap:
        cmap = plt.cm.Blues

    cm = metrics.confusion_matrix(labels, predictions, normalize=normalize)
    
    # plt.gcf().set_size_inches(30, 10)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, round(cm[i, j], 2),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    #plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return plt.gcf()

In [9]:
def plot_roc(
    context,
    y_labels,
    y_probs,
    fpr_label: str = "false positive rate",
    tpr_label: str = "true positive rate",
    title: str = "roc curve",
    legend_loc: str = "best",
):
    """plot roc curves

    TODO:  add averaging method (as string) that was used to create probs, 
    display in legend

    :param context:      the function context
    :param y_labels:     ground truth labels, hot encoded for multiclass  
    :param y_probs:      model prediction probabilities
    :param key:          ("roc") key of plot in artifact store
    :param plots_dir:    ("plots") destination folder relative path to artifact path
    :param fmt:          ("png") plot format
    :param fpr_label:    ("false positive rate") x-axis labels
    :param tpr_label:    ("true positive rate") y-axis labels
    :param title:        ("roc curve") title of plot
    :param legend_loc:   ("best") location of plot legend
    """
    from sklearn import metrics
    import matplotlib.pyplot as plt
    from mlrun.mlutils import gcf_clear
    
    # clear matplotlib current figure
    gcf_clear(plt)

    # draw 45 degree line
    plt.plot([0, 1], [0, 1], "k--")

    # labelling
    plt.xlabel(fpr_label)
    plt.ylabel(tpr_label)
    plt.title(title)
    plt.legend(loc=legend_loc)

    # single ROC or mutliple
    if y_labels.ndim > 2:
        # data accummulators by class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(y_labels[:, :-1].shape[1]):
            fpr[i], tpr[i], _ = metrics.roc_curve(
                y_labels[:, i], y_probs[:, i], pos_label=1
            )
            roc_auc[i] = metrics.auc(fpr[i], tpr[i])
            plt.plot(fpr[i], tpr[i], label=f"class {i}")
    else:
        fpr, tpr, _ = metrics.roc_curve(y_labels, y_probs[:,-1])
        plt.plot(fpr, tpr, label=f"positive class")

    return plt.gcf()

## probabilities

### generate probabilities

In [10]:
def gen_proba(
    context,
    feats,
    labels,
    model,
    score_method,
    plots_dest,
    ntree_limit=None,
    validate_features=True,
    base_margin=None
):
    """ generate predictions and validation stats
    
    :param context:           the function execution context
    :param feats:             validation features array 
    :param labels:            validation ground-truth labels
    :param model:             estimated model
    :param scrore_method:     ("average") multiclass scoring
    :param plots_dest:        destination folder for plot artifacts
    :param ntree_limit:       (None) limit no. trees used in prediction
    :param validate_features: (True) ensure consistent feature names 
                              between model and input data
    :param base_margin:       (None) undefined
    """
    from sklearn import metrics
    from mlrun.artifacts import PlotArtifact
    from mlrun.mlutils import gcf_clear
    from xgboost import XGBClassifier
    import matplotlib.pyplot as plt
    
    ypred = model.predict(feats, False, ntree_limit, validate_features, base_margin)
    
    y_proba = []
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(feats, ntree_limit, validate_features, base_margin)
    ypred_binary = [round(value) for value in y_proba[:,-1]]
    
    average_precision = metrics.average_precision_score(labels, y_proba[:,-1], average=score_method)
    context.log_result(f"avg_precision", average_precision)
    context.log_result(f"rocauc", metrics.roc_auc_score(labels, y_proba[:,-1]))
    context.log_result(f"accuracy_score", float(metrics.accuracy_score(labels, ypred_binary)))
    context.log_result(f"f1_score", metrics.f1_score(labels, ypred_binary, average=score_method))
    
    # ROC plot
    context.log_artifact(PlotArtifact("roc", body=plot_roc(context, labels, y_proba)),
                         local_path=f"{plots_dest}/roc.html")
    gcf_clear(plt)

    body = plot_confusion_matrix(labels, ypred_binary, classes=labels.labels.unique()) 
    context.log_artifact(PlotArtifact("confusion", body=body), local_path=f"{plots_dest}/confusion.html")
    
    return y_proba

### calibrate probabilities

In [11]:
def proba_calibration(model, xcal, ycal):
    """calibrate output probabilities
    
    adapted from https://scikit-learn.org/stable/auto_examples/calibration/plot_calibration_curve.html#sphx-glr-auto-examples-calibration-plot-calibration-curve-py
    """
    import matplotlib.pyplot as plt
    from sklearn.calibration import (CalibratedClassifierCV,
                                     calibration_curve)
    from sklearn.linear_model import LogisticRegression
    from sklearn import metrics
    
    def plot_calibration_curve(name, fig_index):
        """Plot calibration curve for est w/o and with calibration. """
        # Calibrated with isotonic calibration
        isotonic = CalibratedClassifierCV(model, cv="prefit", method='isotonic')

        # Calibrated with sigmoid calibration
        sigmoid = CalibratedClassifierCV(model, cv="prefit", method='sigmoid')

        # Logistic regression with no calibration as baseline
        lr = LogisticRegression(C=1.)

        fig = plt.figure(fig_index, figsize=(10, 10))
        ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
        ax2 = plt.subplot2grid((3, 1), (2, 0))

        ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
        for clf, name in [(lr, 'Logistic'),
                          (model, name),
                          (isotonic, name + ' + Isotonic'),
                          (sigmoid, name + ' + Sigmoid')]:
            clf.fit(xcal, ycal)
            y_pred = clf.predict(xcal)
            if hasattr(clf, "predict_proba"):
                prob_pos = clf.predict_proba(xcal)[:, 1]
            else:  # use decision function
                prob_pos = clf.decision_function(xcal)
                prob_pos = \
                    (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())

            clf_score = metrics.brier_score_loss(ycal, prob_pos, pos_label=1) # pos_label=ycal.max())
            print("%s:" % name)
            print("\tBrier: %1.3f" % (clf_score))
            print("\tPrecision: %1.3f" % metrics.precision_score(ycal, y_pred))
            print("\tRecall: %1.3f" % metrics.recall_score(ycal, y_pred))
            print("\tF1: %1.3f\n" % metrics.f1_score(ycal, y_pred))

            fraction_of_positives, mean_predicted_value = \
                calibration_curve(ycal, prob_pos, n_bins=10)

            ax1.plot(mean_predicted_value, fraction_of_positives, "s-",
                     label="%s (%1.3f)" % (name, clf_score))

            ax2.hist(prob_pos, range=(0, 1), bins=10, label=name,
                     histtype="step", lw=2)

        ax1.set_ylabel("Fraction of positives")
        ax1.set_ylim([-0.05, 1.05])
        ax1.legend(loc="lower right")
        ax1.set_title('Calibration plots  (reliability curve)')

        ax2.set_xlabel("Mean predicted value")
        ax2.set_ylabel("Count")
        ax2.legend(loc="upper center", ncol=2)

        plt.tight_layout()

    # Plot calibration curve for XGBoost
    plot_calibration_curve("XGBoost", 1)

    plt.show()

## train

In [12]:
def train_model(
    context,
    model_type: str,
    dataset,
    label_column: str = "labels",
    sample: int = -1,
    test_size: float = 0.05,
    valid_size: float = 0.75,
    random_state: int = 1,
    model_filename: str = "model",
    models_dest: str = "",
    plots_dest: str = "",
    score_method: str = "micro",
    file_ext: str = "parquet",
    model_pkg_file: str = "",    
) -> None:
    """train an xgboost model.

    :param context:           the function context
    :param model_type:        the model type to train, 'classifier', 'regressor'...
    :param dataset:           ("data") name of raw data file
    :param label_column:      ground-truth (y) labels
    :param sample:            Selects the first n rows, or select a sample
                              starting from the first. If negative <-1, select
                              a random sample
    :param model_filename:    model file filename,
                              points to a directory
    :param test_size:         (0.05) test set size
    :param valid_size:          (0.75) Once the test set has been removed the
                              training set gets this proportion.
    :param random_state:      (1) sklearn rng seed
    :param models_dest:       models subfolder on artifact path
    :param plots_dest:        plot subfolder on artifact path
    :param score_method:      for multiclass classification
    
    :param file_ext:          format for test_set_key hold out data
    :param model_pkg_file:    json model config file                                  
    """
    # deprecate:
    models_dest = models_dest or "models"
    plots_dest = plots_dest or f"plots/{context.name}"
    
    # get a sample from the raw data
    raw, labels, header = get_sample(str(dataset), sample, label_column)
    
    # split the sample into train validate, test and calibration sets:
    (xtr,ytr), (xva,yva), (xte,yte), (xcal, ycal) = get_splits(raw, labels, 4,
                                                               test_size, 
                                                               valid_size, 
                                                               ["labels"],
                                                               random_state)
        
    # get xgboost model and model config
    model, model_config = gen_xgb_model(model_type, context.parameters.items())
    
    # update the model config with training data and callbacks
    model_config["FIT"].update({"X": xtr,"y": ytr.values})
    
    # run the fit
    model.fit(**model_config["FIT"])
    
    # serialize the model
    dump_xgb_model(context, model, "json", models_dest, model_filename)

    # generate predictions
    y_proba = gen_proba(context, xva, yva, model, score_method, plots_dest)
    
    # calibrate probabilities
    y_proba_cal = proba_calibration(model, xcal, ycal)

In [13]:
# nuclio: end-code

### mlconfig

In [14]:
from mlrun import mlconf

In [15]:
mlconf.dbpath = mlconf.dbpath or "./"
mlconf.dbpath

'http://mlrun-api:8080'

In [16]:
vcs_branch = "development"
base_vcs = f"https://raw.githubusercontent.com/mlrun/functions/{vcs_branch}/"

mlconf.hub_url = mlconf.hub_url or base_vcs + f"{name}/function.yaml"
mlconf.hub_url

'/User/repos/functions/{name}/function.yaml'

In [17]:
import os
mlconf.artifact_path = mlconf.artifact_path or f"{os.environ['V3IO_HOME']}/artifacts"
mlconf.artifact_path

'/User/artifacts'

In [18]:
import os
TAG = os.environ["MLRUN_COMMIT"]

### save

In [19]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("xgb_trainer", kind="job", with_doc=True,
                      handler=train_model,
                      image=f"mlrun/ml-models:{TAG}")

# add metadata (for templates and reuse)
fn.spec.default_handler = "train_model"
fn.spec.description = "train any classifier using scikit-learn's API"
fn.metadata.categories = ["models", "classifier"]
fn.metadata.labels = {"author": "yjb"}

fn.save()
fn.export("function.yaml")

[mlrun] 2020-04-28 00:07:25,218 saving function: xgb-trainer, tag: latest
[mlrun] 2020-04-28 00:07:25,321 function spec saved to path: function.yaml


### test function

In [20]:
from mlrun import import_function, mount_v3io, NewTask, run_local

func = import_function("hub://xgb_trainer")

if "V3IO_HOME" in list(os.environ):
    # mlrun on the iguazio platform
    from mlrun import mount_v3io
    fn.apply(mount_v3io())
else:
    # mlrun is setup using the instructions at 
    # https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    fn.apply(mount_pvc("nfsvol", "nfsvol", "/home/joyan/data"))

In [21]:
gpus = False

task_params = {
    "name" : "tasks xgb cpu trainer",
    "params" : {
        "model_type"         : "classifier", # choose regressor, ranker, rfclassifier...
        "num_class"          : 2,  # do not use this when binary
        "CLASS_tree_method"  : "gpu_hist" if gpus else "hist",
        "CLASS_objective"    : "binary:logistic",  # have this chosen by default
        "CLASS_random_state" : 1,
        "sample"             : -1,
        "label_column"       : "labels",
        "test_size"          : 0.10,
        "valid_size"         : 0.75,
        "score_method"       : "weighted",
        "models_dest"        : os.path.join(mlconf.artifact_path, "models"),
        "plots_dest"         : os.path.join(mlconf.artifact_path, "plots"),
    }}

### run remotely

In [22]:
run = fn.run(
    NewTask(**task_params),
    inputs={"dataset"  : os.path.join(mlconf.artifact_path, "classifier-data.csv")})

[mlrun] 2020-04-28 00:07:25,414 starting run tasks xgb cpu trainer uid=e10d57efe7504f1282214dd7621148c9  -> http://mlrun-api:8080
[mlrun] 2020-04-28 00:07:25,688 Job is running in the background, pod: tasks-xgb-cpu-trainer-l4gpm
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
No handles with labels found to put in legend.
[mlrun] 2020-04-28 00:07:33,283 log artifact model at /User/artifacts/e10d57efe7504f1282214dd7621148c9/User/artifacts/models/model.pkl, size: 24364, db: Y
[mlrun] 2020-04-28 00:07:33,717 log art

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...621148c9,0,Apr 28 00:07:29,"<div style=""color: red;"" title=""feature_names mismatch: ['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6', 'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30', 'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36', 'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42', 'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47', 'feat_48', 'feat_49', 'feat_50', 'feat_51', 'feat_52', 'feat_53', 'feat_54', 'feat_55', 'feat_56', 'feat_57', 'feat_58', 'feat_59', 'feat_60', 'feat_61', 'feat_62', 'feat_63', 'labels'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64']\nexpected feat_44, feat_52, feat_37, feat_55, feat_10, feat_3, feat_34, feat_23, feat_43, feat_49, feat_38, feat_36, feat_29, feat_42, feat_9, feat_47, feat_63, feat_51, feat_6, feat_45, feat_62, feat_2, feat_22, labels, feat_40, feat_8, feat_46, feat_12, feat_26, feat_5, feat_60, feat_39, feat_28, feat_59, feat_25, feat_18, feat_57, feat_1, feat_20, feat_0, feat_58, feat_16, feat_30, feat_35, feat_50, feat_56, feat_13, feat_4, feat_19, feat_33, feat_7, feat_61, feat_17, feat_15, feat_21, feat_41, feat_14, feat_31, feat_54, feat_27, feat_11, feat_53, feat_48, feat_24, feat_32 in input data\ntraining data did not have the following fields: f17, f52, f30, f7, f25, f13, f39, f53, f9, f44, f47, f23, f10, f14, f42, f31, f21, f5, f46, f3, f24, f64, f6, f15, f41, f54, f60, f16, f33, f18, f32, f38, f27, f22, f59, f26, f50, f1, f56, f2, f35, f36, f61, f20, f28, f37, f55, f63, f4, f43, f57, f62, f58, f29, f8, f48, f49, f12, f0, f40, f19, f34, f45, f11, f51"">error",tasks xgb cpu trainer,host=tasks-xgb-cpu-trainer-l4gpmkind=jobowner=adminv3io_user=admin,dataset,CLASS_objective=binary:logisticCLASS_random_state=1CLASS_tree_method=histlabel_column=labelsmodel_type=classifiermodels_dest=/User/artifacts/modelsnum_class=2plots_dest=/User/artifacts/plotssample=-1score_method=weightedtest_size=0.1valid_size=0.75,accuracy_score=1.0avg_precision=1.0f1_score=1.0rocauc=1.0,modelrocconfusion


to track results use .show() or .logs() or in CLI: 
!mlrun get run e10d57efe7504f1282214dd7621148c9  , !mlrun logs e10d57efe7504f1282214dd7621148c9 
[mlrun] 2020-04-28 00:07:37,949 run executed, status=error
runtime error: feature_names mismatch: ['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6', 'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30', 'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36', 'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42', 'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47', 'feat_48', 'feat_49', 'feat_50', 'feat_51', 'feat_52', 'feat_53', 'feat_54', 'feat_55', 'feat_56', 'feat_57', 'feat_58', 'feat_59', 'feat_60', 'feat_61', 'feat_62', 'feat_63', 'labels'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7',

RunError: feature_names mismatch: ['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6', 'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12', 'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18', 'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24', 'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30', 'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36', 'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42', 'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47', 'feat_48', 'feat_49', 'feat_50', 'feat_51', 'feat_52', 'feat_53', 'feat_54', 'feat_55', 'feat_56', 'feat_57', 'feat_58', 'feat_59', 'feat_60', 'feat_61', 'feat_62', 'feat_63', 'labels'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38

# tests

WIP

In [ ]:
def test_gen_xgb_model():
    import xgboost
    c, j = gen_xgb_model("rf_classifier", {})
    assert isinstance(c, xgboost.XGBRFClassifier)
test_gen_xgb_model()    

In [ ]:
def test_get_sample():
    from mlrun import mlconf
    r, l, h = get_sample(mlconf.artifact_path+"/breast_cancer.parquet", -1, "labels")
    assert r.shape[0]==l.shape[0]
test_get_sample()

In [ ]:
def test_get_splits():
    from mlrun import mlconf
    r, l, h = get_sample(mlconf.artifact_path+"/classifier-data.csv", -1, "labels")
    (xtr, ytr), (xva, yva), (xte, yte), (xcal, ycal) = get_splits(r, l, 4)

    assert xtr.shape[0]+xva.shape[0]+xte.shape[0]+xcal.shape[0] == r.shape[0]
test_get_splits()

In [ ]:
def test_save_test_set():
    r, l, h = get_sample(mlconf.artifact_path+"/classifier-data.csv", -1, "labels")
    (xtr, ytr), (xva, yva), (xte, yte) = get_splits(r,l)
    from mlrun import get_or_create_ctx
    save_test_set(get_or_create_ctx("test"), xte, yte, h, debug=True)
    import pandas as pd
    # pd.read_parquet()
    # assert
test_save_test_set()

In [ ]:
def test_dump_xgb_model_pickle():
    """
    note that this approach doesn't require information
    about the underlying package, xgboost
    """
    from cloudpickle import load
    from mlrun import mlconf
    
    booster = load(open(mlconf.artifact_path + "/models/xgb-dump.pkl", "rb"))
    assert "booster" in booster.__dict__.keys()

test_dump_xgb_model_pickle()

def test_dump_xgb_save_model():
    import xgboost as xgb
    from mlrun import mlconf

    booster = xgb.Booster()
    booster.load_model(fname=mlconf.artifact_path + "/models/xgb-save_model.pkl")
    assert "booster" in booster.__dict__.keys()

test_dump_xgb_save_model()

def test_dump_xgb_json_save_model():
    import xgboost as xgb
    from mlrun import mlconf

    booster = xgb.Booster()
    booster.load_model(fname=mlconf.artifact_path + "/models/xgb-save_model.json")
    assert "booster" in booster.__dict__.keys()

test_dump_xgb_json_save_model()

def test_gen_proba():
    from cloudpickle import load
    from mlrun import mlconf, get_or_create_ctx
    
    model = load(open(mlconf.artifact_path+'/models/xgb-dump.pkl', 'rb'))
    r, l, h = get_sample(mlconf.artifact_path+"/breast_cancer.parquet", -1, "labels")
    (xtr, ytr), (xva, yva), (xte, yte) = get_splits(r,l)
    y_proba = gen_proba(get_or_create_ctx('test'),
                        xtr, ytr, model, "macro",
                        mlconf.artifact_path+'/plots')
    return y_proba, model

y_proba, model = test_gen_proba()